<a href="https://colab.research.google.com/github/syh0397/Statistic_python/blob/main/4_Statistic_python_Chi_square_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **IMPORT**

In [2]:
import pandas as pd 
import numpy as np 

import os
import sys
import warnings
warnings.filterwarnings('ignore')

import sklearn 
import missingno as msno

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('seaborn')
sns.set(font_scale=2.5) 
# 이 두줄은 본 필자가 항상 쓰는 방법입니다. 
# matplotlib 의 기본 scheme 말고 seaborn scheme 을 세팅하고, 일일이 graph 의 font size 를 지정할 필요 없이 seaborn 의 font_scale 을 사용하면 편합니다.
import scipy 
from scipy import  stats


In [3]:
# Bring in common data from Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/data/Breast Cancer Wisconsin.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [5]:
df.dtypes

id                           int64
diagnosis                   object
radius_mean                float64
texture_mean               float64
perimeter_mean             float64
area_mean                  float64
smoothness_mean            float64
compactness_mean           float64
concavity_mean             float64
concave points_mean        float64
symmetry_mean              float64
fractal_dimension_mean     float64
radius_se                  float64
texture_se                 float64
perimeter_se               float64
area_se                    float64
smoothness_se              float64
compactness_se             float64
concavity_se               float64
concave points_se          float64
symmetry_se                float64
fractal_dimension_se       float64
radius_worst               float64
texture_worst              float64
perimeter_worst            float64
area_worst                 float64
smoothness_worst           float64
compactness_worst          float64
concavity_worst     

### **카이제곱 검정/적합도 검정 (Chi-square Test)/(Goodness of Fit Test)**

카이 제곱 검정은, 
- **관찰된 빈도가 기대되는 빈도와 유의미하게 다른지를 검증하는 통계 검정 방법**이다. 

- 주로 **범주형 자료**로 구성된 데이터 분석에 이용된다. 
- 핵심은 두 범주형 변수가 서로 상관이 있는 지 혹은 독립 관계인지 이다.  
- 참고로 범주형 자료는 categorical data 로, 월 소득 100만원 미만, 이상 등 구간에 대한 자료를 의미한다. 



---

카이제곱 검정의 형태는 다음과 같다. 

 

1. Goodness of fit test : 적합도 검정.  (Pearson의 카이제곱 검정)

적합도 검정이란, 어떤 모집단의 표본이 그 모집단을 대표할 수 있는 지 검정하는 방법으로, 관찰 된 비율 값이 기대값과 같은지 여부를 검정하는 방법이다. 변수는 1개 이다. 

 

2. Test of homogeneity : 동질성 검정.

동질성 검정이란, 두 집단의 분포가 동일한지 검정하는 방법이다.

 

3. Test for independence : 독립성 검정. 

동립성 검정은 두 개 이상의 변수가 독립인지 검정하는 방법이다. 즉, 각 표본들이 관찰 값에 영향을 주는지 여부를 검정하는 방법이다. 

 



---


적합도 검정

- 카이제곱 검정의 종류중 하나로 
- 관측도수와 기대도수의 따른 비율차이가 통계적으로 유의한지 살펴보는 검정


- 관측도수 = 실제 표본조사 빈도
- 기대도수 = 알려진 모집단의 분포에 의해서 예상되는 빈도

그니까 **주어진 데이터의 분포가 예상되는 분포랑 비슷하냐 ?** ? 물어보면 대답할때 쓴다.


---



라고 하고 해보려고 했는데 
;;

범주형 변수가 진단 하나 뿐

(마치 알약 두봉지뿐)

그래서 범주형 변수를 하나 더 만들어 준다. 임의로 
(뭔지 모르지만 일단 막 만들어!!)

In [6]:
def new(x):
  if x >20:
    return 1
  else:
    return 0


In [7]:
df['new'] = df['radius_mean'].apply(new)

In [8]:
df

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32,new
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,NaN,0
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,NaN,1
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,NaN,0
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,NaN,0
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,1.1760,1.2560,7.673,158.70,0.010300,0.02891,0.05198,0.02454,0.01114,0.004239,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,NaN,1
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,0.7655,2.4630,5.203,99.04,0.005769,0.02423,0.03950,0.01678,0.01898,0.002498,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,NaN,1
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,0.4564,1.0750,3.425,48.55,0.005903,0.03731,0.04730,0.01557,0.01318,0.003892,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,NaN,0
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,0.7260,1.5950,5.772,86.22,0.006522,0.06158,0.07117,0.01664,0.02324,0.006185,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,NaN,1


In [9]:
# 크로스탭 쿼리를 통해 빈도교차표를 출력한다. 
X = pd.crosstab(df.new, df.diagnosis, margins= True) 
# Margins = 행과 열의 총합을 표시 할 것인가?
X

diagnosis,B,M,All
new,,,
0,357,167,524
1,0,45,45
All,357,212,569


범주형 변수가 아니더라도 범주형 변수로 바꿔서 이렇게 빈도를 나타내는 교차표를 만들었다. 

ALL 은 총합

- 바꿔말하면 범주형 변수가 아니더라도 가능함 
- 실제 표본조사 빈도(관측도수) 와 알려진 모집단의 분포에 의해서 예상되는 빈도(기대도수)가 통계적으로 유의한지 살펴보자 :)


In [10]:
# 진단과 radius_mean이 20이 넘는지 안넘는지의 열(new)을가지고 통계적으로 유의미한지 보았을때 관찰값
Ob = X.values[1,:2]
# 관측 비율 (여기서는 기존에 알려진 비율이 존재한다고 가정하자)
Pr = np.array([0.7,0.3]) 
n = X.values[1,2]

E = n*Pr 
stats.chisquare(Ob,E)

Power_divergenceResult(statistic=105.0, pvalue=1.221358380729662e-24)

- p밸류가 0.05를 넘어감, 그니까 귀무가설H0을 그대로 살려두고 , H1인 대안가설을 죽임(기각)
- 관측치와 예상치(기대)는 다르다고 말할 충분한 근거가 없다. 

- 관측도수와 기대도수의 차이는 통계적으로 의미 없다~
- 모집단의 분포와 표본의 분포는 차이가 없다. (차이가 있는지 보려고 했으니까)



---

사실 의미가 없는게

적합도 검정 조건

- 범주형 변수 값의 갯수를 알 때 (단순 랜덤 표본에 해당하는 값이어야 함)
- 범주형, 명목형, 연속형 데이터에는 적합하지 않음. 
- 관측된 각 데이터 범주에서 최소 5개의 값이 기대될 정도의 사이즈. 

가 되어야 한다.

물론 이게 아니어도 상관은 없다.

예를들어서 공을 뽑는다고 하자

1.   흰공 3 검은공 4 빨간공 2 
2.   총 9개의 공을 뽑았다.
3.   90개의 공이 30개씩 담겨있었을 것이라고 예상할때
4.   뽑은 공의 색깔에 대한 분포와 
5.   기대하는 분포 1/3 이 같은지 보는 것이다.
6.   딱봐도 다르네....

- 이때 귀무가설 H0 은 변수X(공 색깔)의 관측 분포와 기대분포가 동일하다
- 이때 대안가설 H1 은 변수X(공 색깔)의 관측 분포와 기대분포가 동일하지 않다. 

동일하지 않으니까 ㄱ,,,,귀무가설 기각 




### **카이제곱 검정/독립성 검정 (Chi-square Test)/(Test of Independence)**

독립성검정은 
- 범주형인 두 변수가 서로 연관되어 있는지 여부를 검정
- 연속형 변수들 사이의 관계를 알아보는 상관분석이 있다면, 
- 범주형 변수에는 독립성검정이 있습니다. 

 
예를 들어봅시다. 

**성별과 흡연여부의 관계**를 알고 싶어서 임의로 200명을 추출하여 성별 및 흡연여부를 조사하였습니다. 

```
      흡연	  비흡연	  합계
남성	 46	   33	  79
여성	 25	   96	  121
합계	 71	   129	 200
```

- 귀무가설 : 변수 X성별와 Y흡연유무는 서로 독립이다.
- 대립가설 : 변수 X성별와 Y흡연유무는 서로 독립이 아니다. 

궁금하면 한번 해보시는걸로 

In [11]:
# 새로운 열이 추가된 df를 그대로 써보자

df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32,new
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN,0
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN,1
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN,0
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN,0
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN,1


크로스탭도 그대로 쓰자 .... 범주형 변수가 없잖아 ? 
데이터를 다른걸 쓰든지 해야지

대신 이번에는 margins= False

In [12]:
# 크로스탭 쿼리를 통해 빈도교차표를 출력한다. 
X = pd.crosstab(df.new, df.diagnosis, margins= False) 
# Margins = 행과 열의 총합을 표시 할 것인가?
# 이번에는 하지말자 
X

diagnosis,B,M
new,,
0,357,167
1,0,45


In [13]:
stats.chi2_contingency(X)

(79.39733176671615,
 5.079466303860378e-19,
 1,
 array([[328.76625659, 195.23374341],
        [ 28.23374341,  16.76625659]]))

- 두 변수가 독립적인가 ? : 독립성 검정
- 위에 있는 부분이 카이제곱 검정 통계량
- 그 바로 밑에가 유의 확률 (10의 -19승을 곱해야 하니까 엄청나게 유의하다고 볼 수 있음)
- 1은 자유도 
- 밑에는 각 셀에따른 기대빈도, 기대도수가 나타난다. (잘보면 표랑 비슷)

결과적으로 new 와 diagnosis 는 통계적으로 유의하게  연관성이 있다. 
= 독립적이지 않다. 

### **카이제곱 검정/동질성 검정 (Chi-square Test)/(Test of Homogeneity)**

동질성검정은 독립성검정처럼 변수가 2개입니다. 

독립성검정이 두 변수의 관계를 알기 위해 하는 검정이지만, 

동질성검정은 두 변수의 관계를 알기 위해 하는 검정은 아닙니다. 

- 동질성검정은 한 변수의 요인들에 관심이 있습니다. 
- 요인 보다는 그룹이라고 하는 것이 이해하기 쉽습니다. 
- 각 그룹들이 동질한지 알고 싶은 것입니다. 여기서 동질하다는 것은 확률분포가 같다는 것입니다. 
- 서울 부산, 인천 경기 와 같이 모집단이 다른곳에서 얻은 데이터가 통계적으로 (비율적으로) 유의미한지 보는 것입니다.
- 모집단 다른 두 표본 / 동질적이냐 이질적이냐 ? 

In [14]:
# 크로스탭 쿼리를 통해 빈도교차표를 출력한다. 
X = pd.crosstab(df.new, df.diagnosis, margins= False) 
# Margins = 행과 열의 총합을 표시 할 것인가?
# 이번에는 하지말자 
X

diagnosis,B,M
new,,
0,357,167
1,0,45


빈도교차표를 출력하고


In [15]:
stats.chi2_contingency(X)

(79.39733176671615,
 5.079466303860378e-19,
 1,
 array([[328.76625659, 195.23374341],
        [ 28.23374341,  16.76625659]]))

결과적으로는 귀무가설을 기각하니까 => 차이가 있다 동질적이지 않다.

근데 위에거랑 똑같네 ? 

그럼 독립성 검정과 동질성 검정은 뭐가 다를까? 

 

### **독립성 검정과 동질성 검정의 차이** 

동립성검정과 동질성검정은 변수에 종류에 의해 결정되지 않습니다. 

우리의 '관점'과 '표본 추출 방법'에 의해 결정됩니다. 

'성별'과 '흡연여부'를 두개의 변수로 취급할지 아니면 '흡연 여부' 만을 변수로 보고, 성별은 비교 대상이 되는 '그룹'으로 해석할지는 우리가 결정하는 것입니다. 

 

1. 만약 '성별'과 '흡연여부'를 두개의 변수로 보고 독립성검정을 하겠다고 결정했다면, 
표본추출은 전체집단 하나만 추출합니다.

 

2. 만약 '흡연 여부'만을 변수로 보고 남/녀 그룹의 동질성을 비교하고 싶다면, 남자 그룹과 여자그룹의 표본을 각각 추출합니다. 각 표본에서 흡연 여부를 조사합니다. 


https://hsm-edu.tistory.com/1214?category=934374